In [1]:
import geopandas as gpd
import pandas as pd
import os
from collections import Counter
from datetime import date
import openpyxl
from win32com.client import Dispatch
from wdtools import *

In [4]:
len(read_list('Set001'))+len(read_list('Set002'))

23

In [5]:
sets = list(map(lambda x: 'Set00'+x, ['1', '2', '3']))

In [6]:
nm2add = [0, 1420, 2143]

In [7]:
dat = []

In [8]:
for setid in sets:
    df = combine_wd_tables(setID=setid, nm_to_add=nm2add[sets.index(setid)])
    df.loc[:, 'SetID'] = setid
    dat.append(df)

In [9]:
data=pd.concat(dat)

In [10]:
data.shape

(2878, 26)

In [11]:
data.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [12]:
data.loc[data.county=='TIllamook', 'county'] = 'Tillamook'

In [13]:
ndf = data.groupby('county', as_index=False)['wetdet_delin_number'].nunique()

In [14]:
ndf.rename(columns={'wetdet_delin_number': 'total'}, inplace=True)

In [15]:
qaqc = []

In [16]:
qaqcdf = pd.DataFrame(columns=['SetID', 'Counties', 'Total Count', 'Total QAQC Count', 
                               'Partial and Mythical Count', 'Unmatched Count'])

In [17]:
unmt = []

In [18]:
for setid in sets:
    print(setid)
    counties = data[data.SetID==setid]['county'].unique()
    print(counties)
    N = data[data.SetID==setid]['wetdet_delin_number'].nunique()
    print(N)
    matched = gpd.read_file(os.path.join(inpath + '\\output\\matched\\', f'matched_records_{setid}_edited.shp'))
    a = list(matched[matched.notes.notnull()].wdID.unique())
    unmatched = pd.read_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setid}_2.csv'))
    b = list(unmatched.wetdet_delin_number.unique())
    unmatched_wd = [x for x in b if x not in a]
    unmatched3 = unmatched[unmatched.wetdet_delin_number.isin(unmatched_wd)]
    unmatched3.to_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setid}_3.csv'), index=False)
    unmt.append(unmatched_wd)
    qaqc.append(unique(a+b))
    print(f"""partial and mythical count - {len(a)}, 
          unmatched count - {len(unmatched_wd)}
          total QAQC count - {len(unique(a+b))}""")
    qaqcdf = qaqcdf.append({'SetID': setid,
                            'Counties': ', '.join(list(counties)),
                            'Total Count': N,
                            'Total QAQC Count': len(unique(a+b)), 
                            'Partial and Mythical Count': len(a), 
                            'Unmatched Count': len(unmatched_wd)}, ignore_index=True)

Set001
['Clatsop' 'Coos' 'Deschutes' 'Harney' 'Jackson' 'Josephine' 'Lake'
 'Marion']
1003
partial and mythical count - 264, 
          unmatched count - 14
          total QAQC count - 278
Set002
['Baker' 'Benton' 'Clackamas' 'Morrow']
434
partial and mythical count - 138, 
          unmatched count - 14
          total QAQC count - 152
Set003
['Columbia' 'Grant' 'Klamath' 'Lincoln' 'Tillamook']
433
partial and mythical count - 127, 
          unmatched count - 11
          total QAQC count - 138


In [19]:
file=os.path.join(inpath, 'reporting', f'SetStatusReport{str(date.today()).replace("-", "")}.xlsx')

In [20]:
qaqcdf.to_excel(file, index=False)

In [21]:
reformat(file)

Removed format
Autofitted columns...


In [22]:
qaqc = unique(np.concatenate(qaqc))

In [23]:
len(qaqc)

567

In [24]:
df = data[['wetdet_delin_number', 'county']]

In [25]:
wdcnt_dict = df.set_index('wetdet_delin_number').to_dict(orient='dict')['county']

In [26]:
qaqc_df = count_lst_ele(qaqc, 'QAQC')

In [27]:
mapped = []

In [28]:
for setid in ['Set001', 'Set002']:
    mapped.append(read_list(setid))

In [29]:
mapped = unique(np.concatenate(mapped))

In [47]:
len(mapped)

23

In [30]:
mapped

['WD2017-0347',
 'WD2018-0124',
 'WD2018-0217',
 'WD2018-0306',
 'WD2018-0386',
 'WD2018-0419',
 'WD2018-0522',
 'WD2018-0657',
 'WD2019-0124',
 'WD2019-0231',
 'WD2019-0338',
 'WD2019-0431',
 'WD2019-0544',
 'WD2020-0067',
 'WD2020-0391',
 'WD2020-0536',
 'WD2020-0668',
 'WD2020-0677',
 'WD2021-0162',
 'WD2021-0252',
 'WD2021-0559',
 'WD2021-0616',
 'WD2021-0703']

In [31]:
colnm='QAQC_comp'

In [32]:
mapped_df = count_lst_ele(mapped, colnm)

In [33]:
out = ndf.merge(qaqc_df, on='county', how='left')

In [34]:
out1 = out.merge(mapped_df, on='county', how='left')

In [35]:
sel = out1.county.isin(['Columbia', 'Grant', 'Klamath', 'Lincoln', 'Tillamook'])

In [36]:
out1.loc[sel, colnm] = out1.loc[sel, 'QAQC'] - [1, 0, 5, 0, 5]

In [37]:
sel1 = out1.QAQC_comp.isnull()

In [38]:
out1.loc[sel1, colnm] = 0

In [39]:
out1[colnm] = out1[colnm].apply(np.int64)

In [40]:
out1['QAQC_pct'] = out1.apply(lambda row: str(round(row.QAQC/row.total*100,1))+('%'), axis=1)

In [41]:
out1['QAQC_comp_pct'] = out1.apply(lambda row: str(round(row.QAQC_comp/row.QAQC*100,1))+('%'), axis=1)

In [42]:
out1

,county,total,QAQC,QAQC_comp,QAQC_pct,QAQC_comp_pct
0,Baker,38,12,0,31.6%,0.0%
1,Benton,91,30,1,33.0%,3.3%
2,Clackamas,275,96,3,34.9%,3.1%
3,Clatsop,137,41,0,29.9%,0.0%
4,Columbia,93,33,32,35.5%,97.0%
5,Coos,120,27,3,22.5%,11.1%
6,Deschutes,172,39,5,22.7%,12.8%
7,Grant,23,6,6,26.1%,100.0%
8,Harney,13,11,2,84.6%,18.2%
9,Jackson,221,51,4,23.1%,7.8%


In [43]:
out1.columns = ['County', 'Total Count', 'QAQC Count', 'QAQC Completed Count', 
                'QAQC Count Percentage', 'QAQC Completed Count Percentage']

In [44]:
file=os.path.join(inpath, 'reporting', f'StatusReport{str(date.today()).replace("-", "")}.xlsx')

In [45]:
out1.to_excel(file, index=False)

In [46]:
reformat(file)

Removed format
Autofitted columns...
